In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1557200512616_0005,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark.sql("show tables in customer_complaints").show(5,False)

+-------------------+-----------------------+-----------+
|database           |tableName              |isTemporary|
+-------------------+-----------------------+-----------+
|customer_complaints|cust_complaints_all    |false      |
|customer_complaints|cust_complaints_all_raw|false      |
+-------------------+-----------------------+-----------+

In [3]:
df = spark.sql("select * from customer_complaints.cust_complaints_all")

In [4]:
df.cache()
df.count()

1274208

In [5]:
df.printSchema()

root
 |-- date_received: string (nullable = true)
 |-- product: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- consumer_complaint_narrative: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- timely_response: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- complaint_id: string (nullable = true)

In [6]:
dateFormat = "MM/dd/yy"
df1 = df.withColumn("DateRecieved",to_date(col("date_received"), dateFormat))

In [7]:
dfYear = df1.withColumn(
  "YearRecieved",
  year(col("DateRecieved")))

In [8]:
dfMonth = dfYear.withColumn(
  "MonthRecieved",
  month(col("DateRecieved")))

In [9]:
dfzip = dfMonth.withColumn('Zip', df.zip_code.substr(1, 3))

In [10]:
df = dfzip.withColumn('Region', df.zip_code.substr(1, 1))

In [11]:
dfcomplaints = df.where(col("Region") != "N")
dfcomplaints = dfcomplaints.where(col("Region") != "(")
dfcomplaints = dfcomplaints.where(col("Region") != "-")
dfcomplaints = dfcomplaints.where(col("Region") != "*")

In [12]:
dfcomplaints.createOrReplaceTempView("ComplaintsData")

In [13]:
dfYear.groupBy("YearRecieved")\
.count()\
.sort("YearRecieved", ascending= True)\
.show()

+------------+------+
|YearRecieved| count|
+------------+------+
|        2011|  2536|
|        2012| 72373|
|        2013|108218|
|        2014|153047|
|        2015|168487|
|        2016|191473|
|        2017|242975|
|        2018|257378|
|        2019| 77721|
+------------+------+

In [14]:
censusDF = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("s3://finalcomplaintdataset/Census Data.csv")

In [15]:
censusDF.createOrReplaceTempView("CensusData")

In [16]:
print(censusDF.show(5))
print(censusDF.printSchema())

+-----------+------+---+----------+--------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|      State|Region|Abv|    Census|Estimates Base|    2010|    2011|    2012|    2013|    2014|    2015|    2016|    2017|    2018|
+-----------+------+---+----------+--------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|   .Alabama|     3| AL| 4,779,736|     4,780,138| 4785448| 4798834| 4815564| 4830460| 4842481| 4853160| 4864745| 4875120| 4887871|
|    .Alaska|     9| AK|   710,231|       710,249|  713906|  722038|  730399|  737045|  736307|  737547|  741504|  739786|  737438|
|   .Arizona|     8| AZ| 6,392,017|     6,392,288| 6407774| 6473497| 6556629| 6634999| 6733840| 6833596| 6945452| 7048876| 7171646|
|  .Arkansas|     7| AR| 2,915,918|     2,916,028| 2921978| 2940407| 2952109| 2959549| 2967726| 2978407| 2990410| 3002997| 3013825|
|.California|     9| CA|37,253,956|    37,254,523|37320903|37641823|37960782

In [17]:
spark.sql("select Region, YearRecieved, count(*) as Count,  round(count(*) * 100.0/ sum(count(*)) over (), 2) as percentage \
                       from complaintsdata group by Region, YearRecieved order by Region asc, YearRecieved asc").show(100)

+------+------------+-----+----------+
|Region|YearRecieved|Count|percentage|
+------+------------+-----+----------+
|     0|        2011|  261|      0.02|
|     0|        2012| 7414|      0.64|
|     0|        2013| 9852|      0.85|
|     0|        2014|13344|      1.15|
|     0|        2015|12568|      1.08|
|     0|        2016|13258|      1.14|
|     0|        2017|14996|      1.29|
|     0|        2018|15080|      1.30|
|     0|        2019| 4300|      0.37|
|     1|        2011|  310|      0.03|
|     1|        2012| 8297|      0.72|
|     1|        2013|11927|      1.03|
|     1|        2014|16517|      1.42|
|     1|        2015|16853|      1.45|
|     1|        2016|18429|      1.59|
|     1|        2017|20999|      1.81|
|     1|        2018|23659|      2.04|
|     1|        2019| 7098|      0.61|
|     2|        2011|  267|      0.02|
|     2|        2012| 8030|      0.69|
|     2|        2013|12320|      1.06|
|     2|        2014|17551|      1.51|
|     2|        2015|1765

In [19]:
%%sql -q -n 100 -o RegionCount 
select Region, count(*) as Count from complaintsdata group by Region

In [20]:
%%local RegionCount.head(10)

Output()

In [21]:
RegionDF = dfcomplaints.groupBy("Region").agg(count("complaint_id").alias("No-of-Complaints"))
RegCenDF = censusDF.groupBy("Region").agg(sum("2018").alias("Estimated-Population"))
joinType = "inner"
joinExpression = RegionDF["Region"] == censusDF['Region']
RegionDF.join(RegCenDF, joinExpression, joinType).sort("Estimated-Population").show()

+------+----------------+------+--------------------+
|Region|No-of-Complaints|Region|Estimated-Population|
+------+----------------+------+--------------------+
|     5|           32050|     5|            17285509|
|     8|           76790|     8|            23490080|
|     6|           70368|     6|            23708305|
|     0|           91073|     0|            23761810|
|     2|          131195|     2|            32536437|
|     4|           83160|     4|            32845637|
|     1|          124089|     1|            33316440|
|     7|          125492|     7|            40318727|
|     3|          217196|     3|            46463211|
|     9|          208717|     9|            53441278|
+------+----------------+------+--------------------+

In [22]:
%%sql -q -n 100 -o StateCount 
select State, count(*) as Count 
from complaintsdata group by State order by Count desc limit 10

In [23]:
%%local StateCount.head(10)

Output()

In [24]:
StateDF = dfcomplaints.groupBy("State").agg(count("complaint_id").alias("No-of-Complaints"))
stateCensDF = censusDF.withColumn("CA", col("Abv") == "CA")
stateCensDF = stateCensDF.withColumn("FL",col("Abv") == "FL")
stateCensDF = stateCensDF.withColumn("TX",col("Abv") == "TX")
stateCensDF = stateCensDF.withColumn("NY",col("Abv") == "NY")
stateCensDF = stateCensDF.withColumn("GA",col("Abv") == "GA")
stateCensDF = stateCensDF.withColumn("IL",col("Abv") == "IL")
stateCensDF = stateCensDF.withColumn("NJ",col("Abv") == "NJ")
stateCensDF = stateCensDF.withColumn("PA",col("Abv") == "PA")
stateCensDF = stateCensDF.withColumn("NC",col("Abv") == "NC")
stateCensDF = stateCensDF.withColumn("OH",col("Abv") == "OH")
StateCenDF = stateCensDF.select("Abv", "2018").where("CA or FL or TX or NY or GA or NJ or PA or NC or OH")


In [25]:
joinType = "inner"
joinExpression = StateDF["State"] == StateCenDF['Abv']
StateDF.join(StateCenDF, joinExpression, joinType).sort("2018").show()

+-----+----------------+---+--------+
|State|No-of-Complaints|Abv|    2018|
+-----+----------------+---+--------+
|   NJ|           43214| NJ| 8908520|
|   NC|           36349| NC|10383620|
|   GA|           63036| GA|10519475|
|   OH|           34829| OH|11689442|
|   PA|           38621| PA|12807060|
|   NY|           79812| NY|19542209|
|   FL|          117108| FL|21299325|
|   TX|           99956| TX|28701845|
|   CA|          168694| CA|39557045|
+-----+----------------+---+--------+

In [26]:
spark.sql("select company, count(*) as Count, round(count(*) * 100.0/ sum(count(*)) over (),2) as percentage \
from complaintsdata \
group by company \
order by Count desc limit 10").show(10,False)

+--------------------------------------+------+----------+
|company                               |Count |percentage|
+--------------------------------------+------+----------+
|EQUIFAX, INC.                         |104879|9.04      |
|Experian Information Solutions Inc.   |94296 |8.13      |
|TRANSUNION INTERMEDIATE HOLDINGS, INC.|87084 |7.51      |
|BANK OF AMERICA, NATIONAL ASSOCIATION |77420 |6.67      |
|WELLS FARGO & COMPANY                 |65817 |5.67      |
|JPMORGAN CHASE & CO.                  |55759 |4.81      |
|CITIBANK, N.A.                        |44860 |3.87      |
|CAPITAL ONE FINANCIAL CORPORATION     |31361 |2.70      |
|OCWEN LOAN SERVICING LLC              |26266 |2.26      |
|Navient Solutions, LLC.               |25304 |2.18      |
+--------------------------------------+------+----------+

In [27]:
spark.sql("select issue, count(*) as Count, round(count(*) * 100.0/ sum(count(*)) over (),2) as percentage \
from complaintsdata \
where company = 'EQUIFAX, INC.' \
group by issue \
order by Count desc limit 10").show(10, False)

+--------------------------------------------------------------------------------+-----+----------+
|issue                                                                           |Count|percentage|
+--------------------------------------------------------------------------------+-----+----------+
|Incorrect information on credit report                                          |32521|31.01     |
|Incorrect information on your report                                            |31155|29.71     |
|Problem with a credit reporting company's investigation into an existing problem|12401|11.82     |
|Improper use of your report                                                     |10082|9.61      |
|Credit reporting company's investigation                                        |5733 |5.47      |
|Unable to get credit report/credit score                                        |4192 |4.00      |
|Unable to get your credit report or credit score                                |1652 |1.58      |


In [28]:
spark.sql("select issue, count(*) as Count, round(count(*) * 100.0/ sum(count(*)) over (),2) as percentage \
from complaintsdata \
group by issue \
order by Count desc limit 10").show(10, False)

+--------------------------------------------------------------------------------+------+----------+
|issue                                                                           |Count |percentage|
+--------------------------------------------------------------------------------+------+----------+
|Incorrect information on your report                                            |118363|10.20     |
|Loan modification,collection,foreclosure                                        |108392|9.34      |
|Incorrect information on credit report                                          |97432 |8.40      |
|Loan servicing, payments, escrow account                                        |72909 |6.28      |
|Cont'd attempts collect debt not owed                                           |56404 |4.86      |
|Problem with a credit reporting company's investigation into an existing problem|44187 |3.81      |
|Attempts to collect debt not owed                                               |36973 |3.

In [29]:
%%sql -q -n 100 -o ProductCount 
select product, count(*) as Count 
from complaintsdata group by product limit 10

In [30]:
%%local ProductCount.head(10)

Output()

In [31]:
%%sql -q -n 100 -o TimelyCount 
select timely_response, count(*) as Count 
from complaintsdata group by timely_response order by Count desc

In [32]:
%%local TimelyCount.head(10)

Output()

In [33]:
spark.sql("select company, count(*) as Count \
from complaintsdata \
where timely_response = 'No' \
group by company \
order by Count desc limit 10").show(10,False)

+-------------------------------------+-----+
|company                              |Count|
+-------------------------------------+-----+
|WELLS FARGO & COMPANY                |2894 |
|BANK OF AMERICA, NATIONAL ASSOCIATION|1569 |
|EQUIFAX, INC.                        |1464 |
|OCWEN LOAN SERVICING LLC             |525  |
|Colony Brands, Inc.                  |359  |
|CITIBANK, N.A.                       |352  |
|Mobiloans, LLC                       |337  |
|Southwest Credit Systems, L.P.       |271  |
|Midwest Recovery Systems             |207  |
|Residential Credit Solutions, Inc.   |171  |
+-------------------------------------+-----+

In [34]:
spark.sql("select company, product, count(*) as Count \
from complaintsdata \
where timely_response = 'No' and company  = 'WELLS FARGO & COMPANY'\
group by company, product \
order by Count desc limit 10").show(10,False)

+---------------------+----------------------------------------------------------------------------+-----+
|company              |product                                                                     |Count|
+---------------------+----------------------------------------------------------------------------+-----+
|WELLS FARGO & COMPANY|Bank account or service                                                     |1379 |
|WELLS FARGO & COMPANY|Consumer Loan                                                               |369  |
|WELLS FARGO & COMPANY|Credit card                                                                 |293  |
|WELLS FARGO & COMPANY|Checking or savings account                                                 |177  |
|WELLS FARGO & COMPANY|Debt collection                                                             |153  |
|WELLS FARGO & COMPANY|Vehicle loan or lease                                                       |143  |
|WELLS FARGO & COMPANY|Mortgage      

In [35]:
%%sql -q -n 100 -o SubmitCount 
select submitted_via, count(*) as Count 
from complaintsdata group by submitted_via order by Count desc

In [36]:
%%local SubmitCount.head(10)

Output()

In [37]:
spark.sql("select DateRecieved, count(*) as Count, round(count(*) * 100.0/ sum(count(*)) over (),2) as percentage \
from complaintsdata \
group by DateRecieved \
order by Count desc limit 10").show(10)

+------------+-----+----------+
|DateRecieved|Count|percentage|
+------------+-----+----------+
|  2017-09-08| 3118|      0.27|
|  2017-09-09| 2397|      0.21|
|  2017-01-19| 1808|      0.16|
|  2017-01-20| 1432|      0.12|
|  2017-09-13| 1365|      0.12|
|  2018-04-05| 1154|      0.10|
|  2017-09-12| 1072|      0.09|
|  2018-04-10| 1041|      0.09|
|  2018-04-24| 1017|      0.09|
|  2017-09-14|  988|      0.09|
+------------+-----+----------+

In [38]:
spark.sql("select concat(YearRecieved,'/', MonthRecieved) as YearMonth, count(*) as Count, round(count(*) * 100.0/ sum(count(*)) over (),2) as percentage \
from complaintsdata \
group by YearRecieved, MonthRecieved \
order by Count desc limit 10").show(10)

+---------+-----+----------+
|YearMonth|Count|percentage|
+---------+-----+----------+
|   2017/9|23754|      2.05|
|   2018/4|21466|      1.85|
|   2018/3|20714|      1.79|
|   2018/1|20493|      1.77|
|   2019/3|19957|      1.72|
|   2018/5|19444|      1.68|
|   2018/2|19180|      1.65|
|  2018/10|18801|      1.62|
|   2018/8|18765|      1.62|
|   2017/8|18678|      1.61|
+---------+-----+----------+

In [39]:
MortgageDF = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("s3://finalcomplaintdataset/Mortgage Database.csv")

In [40]:
MortgageDF.printSchema()

root
 |-- HUD PROJECT NUMBER: integer (nullable = true)
 |-- PREMISE ID: integer (nullable = true)
 |-- PROPERTY NAME: string (nullable = true)
 |-- PROPERTY STREET: string (nullable = true)
 |-- PROPERTY CITY: string (nullable = true)
 |-- PROPERTY STATE: string (nullable = true)
 |-- PROPERTY ZIP: integer (nullable = true)
 |-- UNITS: integer (nullable = true)
 |-- INITIAL ENDORSEMENT DATE: string (nullable = true)
 |-- FINAL ENDORSEMENT DATE: string (nullable = true)
 |-- ORIGINAL MORTGAGE AMOUNT: string (nullable = true)
 |-- FIRST PAYMENT DATE: string (nullable = true)
 |-- MATURITY DATE: string (nullable = true)
 |-- TERM IN MONTHS: integer (nullable = true)
 |-- INTEREST RATE: double (nullable = true)
 |-- CURRENT PRINCIPAL AND INTEREST: string (nullable = true)
 |-- AMORTIZED PRINCIPAL BALANCE: string (nullable = true)
 |-- HOLDER NAME: string (nullable = true)
 |-- HOLDER CITY: string (nullable = true)
 |-- HOLDER STATE: string (nullable = true)
 |-- SERVICER NAME: string (nul

In [41]:
containsWellsFargo = instr(col("company"), "WELLS FARGO") >= 1
containsMortgage = instr(col("product"), "Mortgage") >= 1
containsYear = instr(col("YearRecieved"), "2018") >= 1

WellsFargoMortgage = dfcomplaints.withColumn("WellsFargoMortgage", containsWellsFargo & containsMortgage & containsYear)\
.where("WellsFargoMortgage")\
.select(["company","product","sub_product","state", "complaint_id"]).show(3, False)

+---------------------+--------+--------------------------+-----+------------+
|company              |product |sub_product               |state|complaint_id|
+---------------------+--------+--------------------------+-----+------------+
|WELLS FARGO & COMPANY|Mortgage|Conventional home mortgage|GA   |2870272     |
|WELLS FARGO & COMPANY|Mortgage|Conventional home mortgage|IL   |2869468     |
|WELLS FARGO & COMPANY|Mortgage|Conventional home mortgage|OH   |2868867     |
+---------------------+--------+--------------------------+-----+------------+
only showing top 3 rows

In [42]:
dfcomplaints.withColumn("WellsFargoMortgage", containsWellsFargo & containsMortgage & containsYear)\
.where("WellsFargoMortgage").groupBy("state").agg(count("complaint_id").alias("No-of-Complaints"))\
.sort("No-of-Complaints",ascending = False).show(10)

+-----+----------------+
|state|No-of-Complaints|
+-----+----------------+
|   CA|             454|
|   FL|             259|
|   NY|             225|
|   TX|             183|
|   NJ|             150|
|   GA|             121|
|   PA|             118|
|   VA|             100|
|   MD|              91|
|   IL|              80|
+-----+----------------+
only showing top 10 rows

In [43]:
containsWellsFargoM = instr(col("SERVICER NAME"), "WELLS FARGO") >= 1

MortgageDF.withColumn("WellsFargoMortgage", containsWellsFargoM ).where("WellsFargoMortgage")\
.groupBy("SERVICER STATE").agg(count("HUD PROJECT NUMBER").alias("No-of-Mortgages")).show(5)

+--------------+---------------+
|SERVICER STATE|No-of-Mortgages|
+--------------+---------------+
|            VA|            984|
|            WI|              7|
+--------------+---------------+